In [1]:
import pandas as pd
import pickle

In [3]:
with open('capstone_1/beerSearches/beer_search_batch_0.pkl', 'rb') as f:
    batch_0 = pickle.load(f)
print(type(batch_0), len(batch_0))

<class 'pandas.core.frame.DataFrame'> 1


In [4]:
batch_0.shape

(1, 85)

ahhh, the horizontal df of batch_0

In [7]:
data = batch_0.iloc[0,0]
type(data)

dict

In [9]:
data['response'].keys()

dict_keys(['message', 'time_taken', 'brewery_id', 'search_type', 'type_id', 'search_version', 'found', 'offset', 'limit', 'term', 'parsed_term', 'beers', 'homebrew', 'breweries'])

In [15]:
item_df = pd.io.json.json_normalize(data['response']['beers']['items'])

In [17]:
item_df.columns

Index(['beer.auth_rating', 'beer.beer_abv', 'beer.beer_description',
       'beer.beer_ibu', 'beer.beer_label', 'beer.beer_name', 'beer.beer_slug',
       'beer.beer_style', 'beer.bid', 'beer.created_at', 'beer.in_production',
       'beer.wish_list', 'brewery.brewery_active', 'brewery.brewery_id',
       'brewery.brewery_label', 'brewery.brewery_name',
       'brewery.brewery_page_url', 'brewery.brewery_slug',
       'brewery.brewery_type', 'brewery.contact.facebook',
       'brewery.contact.instagram', 'brewery.contact.twitter',
       'brewery.contact.url', 'brewery.country_name',
       'brewery.location.brewery_city', 'brewery.location.brewery_state',
       'brewery.location.lat', 'brewery.location.lng', 'checkin_count',
       'have_had', 'your_count'],
      dtype='object')

In [18]:
batch_0_flat = pd.concat([pd.io.json.json_normalize(batch_0.iloc[0,col]['response']['beers']['items'])
                         for col in range(batch_0.shape[1])])

In [19]:
batch_0_flat.shape

(4142, 31)

#### On to the vertical df's

In [29]:
with open('capstone_1/beerSearches/beer_search_batch_1.pkl','rb') as f:
    b = pickle.load(f)
b.columns

Index(['meta', 'notifications', 'response'], dtype='object')

In [30]:
b.loc[0,'response'].keys()

dict_keys(['message', 'time_taken', 'brewery_id', 'search_type', 'type_id', 'search_version', 'found', 'offset', 'limit', 'term', 'parsed_term', 'beers', 'homebrew', 'breweries'])

In [31]:
# now flatten the vertical structures
def flat(row_df_pkl):
    with open(row_df_pkl, 'rb') as f:
        batch = pickle.load(f)
    return pd.concat([pd.io.json.json_normalize(
                        batch.loc[row,'response']['beers']['items'])
                         for row in range(batch.shape[0])])

In [24]:
from glob import glob
beersearches = glob('capstone_1/beerSearches/*.pkl')

In [25]:
beersearches

['capstone_1/beerSearches/beer_search_batch_7.pkl',
 'capstone_1/beerSearches/beer_search_batch_20.pkl',
 'capstone_1/beerSearches/beer_search_batch_21.pkl',
 'capstone_1/beerSearches/beer_search_batch_6.pkl',
 'capstone_1/beerSearches/beer_search_batch_4.pkl',
 'capstone_1/beerSearches/beer_search_batch_22.pkl',
 'capstone_1/beerSearches/beer_search_batch_5.pkl',
 'capstone_1/beerSearches/beer_search_batch_1.pkl',
 'capstone_1/beerSearches/beer_search_batch_0.pkl',
 'capstone_1/beerSearches/beer_search_batch_2.pkl',
 'capstone_1/beerSearches/beer_search_batch_19.pkl',
 'capstone_1/beerSearches/beer_search_batch_18.pkl',
 'capstone_1/beerSearches/beer_search_batch_3.pkl',
 'capstone_1/beerSearches/beer_search_batch_15.pkl',
 'capstone_1/beerSearches/beer_search_batch_14.pkl',
 'capstone_1/beerSearches/beer_search_batch_16.pkl',
 'capstone_1/beerSearches/beer_search_batch_17.pkl',
 'capstone_1/beerSearches/beer_search_batch_8.pkl',
 'capstone_1/beerSearches/beer_search_batch_13.pkl',
 '

In [26]:
# get rid of batch_0, the horizontal one
beersearches = sorted(beersearches)
beersearches.pop(0)

'capstone_1/beerSearches/beer_search_batch_0.pkl'

In [33]:
# flatten all
flats = [flat(batch) for batch in beersearches]

len(flats)

22

In [35]:
df = pd.concat([batch_0_flat] + flats, axis=0, ignore_index=True)

df.shape

(102765, 31)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102765 entries, 0 to 102764
Data columns (total 31 columns):
beer.auth_rating                  102765 non-null int64
beer.beer_abv                     102765 non-null float64
beer.beer_description             102765 non-null object
beer.beer_ibu                     102765 non-null int64
beer.beer_label                   102765 non-null object
beer.beer_name                    102765 non-null object
beer.beer_slug                    102765 non-null object
beer.beer_style                   102765 non-null object
beer.bid                          102765 non-null int64
beer.created_at                   102765 non-null object
beer.in_production                102765 non-null int64
beer.wish_list                    102765 non-null bool
brewery.brewery_active            102765 non-null int64
brewery.brewery_id                102765 non-null int64
brewery.brewery_label             102765 non-null object
brewery.brewery_name              102765 

In [38]:
keepcols = ['beer.beer_abv', 'beer.beer_description', 'beer.beer_ibu',
            'beer.beer_name', 'beer.beer_style', 'beer.bid',
            'brewery.brewery_id', 'brewery.brewery_name',
            'brewery.brewery_type', 'brewery.country_name',
            'brewery.location.brewery_city', 'brewery.location.brewery_state',
            'brewery.location.lat', 'brewery.location.lng', 'checkin_count'
           ]

In [39]:
df = df[keepcols]
df.head()

,beer.beer_abv,beer.beer_description,beer.beer_ibu,beer.beer_name,beer.beer_style,beer.bid,brewery.brewery_id,brewery.brewery_name,brewery.brewery_type,brewery.country_name,brewery.location.brewery_city,brewery.location.brewery_state,brewery.location.lat,brewery.location.lng,checkin_count
0,5.6,"Our most popular beer, Sierra Nevada Pale Ale ...",38,Pale Ale,Pale Ale - American,6284,1142,Sierra Nevada Brewing Co.,Regional Brewery,United States,Chico,CA,39.7242,-121.815,1021656
1,7.2,Sierra Nevada Torpedo Ale is a big American IP...,65,Torpedo Extra IPA,IPA - American,4997,1142,Sierra Nevada Brewing Co.,Regional Brewery,United States,Chico,CA,39.7242,-121.815,664063
2,6.2,Hop Hunter IPA harnesses the complex flavors o...,60,Hop Hunter IPA,IPA - American,568188,1142,Sierra Nevada Brewing Co.,Regional Brewery,United States,Chico,CA,39.7242,-121.815,316738
3,6.8,"The long, cold nights of winter are a little b...",65,Celebration Fresh Hop IPA,IPA - American,16630,1142,Sierra Nevada Brewing Co.,Regional Brewery,United States,Chico,CA,39.7242,-121.815,289647
4,6.7,"As brewers, we get the privilege to sample our...",40,Hazy Little Thing,IPA - New England,2412786,1142,Sierra Nevada Brewing Co.,Regional Brewery,United States,Chico,CA,39.7242,-121.815,276413


In [40]:
df.to_csv('capstone_1/untapped_brewername_search_df.csv')